In [0]:
import numpy as np
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.15.0


In [0]:
# load train and test dataset

(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28*28*1))
testX = testX.reshape((testX.shape[0], 28*28*1))
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)

4423680/4422102 [==============================] - 1s 0us/step


In [0]:
trainX.shape

(60000, 784)

In [0]:
print('trainX.shape',trainX.shape)
print('trainY.shape',trainY.shape)
print('testX.shape',testX.shape)
print('testY.shape',testY.shape)

trainX.shape (60000, 784)
trainY.shape (60000, 10)
testX.shape (10000, 784)
testY.shape (10000, 10)


In [0]:
# scale pixels
# convert from integers to floats
train_norm = trainX.astype('float32')
test_norm = testX.astype('float32')
# normalize to range 0-1
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0

In [0]:
train_norm.shape

(60000, 784)

In [0]:
# Parameters
learning_rate = 0.001
training_epochs = 2000
cost_history = np.empty(shape=[1],dtype=float)
n_dim = train_norm.shape[1]
n_class = 10 
batch_size = 100
display_step = 1

In [0]:
n_dim

784

In [0]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_hidden_3 = 256 # 1st layer number of features
n_hidden_4 = 256 # 2nd layer number of features

In [0]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_dim])
y_ = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))

In [0]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>

In [0]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer [1] with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer [2] with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer [3] with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer [4] with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.sigmoid(layer_4)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

In [0]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, n_class]))
}
biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}

In [0]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [0]:
# Define loss and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost_function)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# Initializing the variables
init = tf.global_variables_initializer()
#create an empty list to store the cost history and accuracy history
#cost_history = []
accuracy_history = []

In [0]:
# Launch the graph
with tf.Session() as sess:
    for epoch in range(training_epochs):
        sess.run(tf.global_variables_initializer())
        sess.run(optimizer,feed_dict={x:train_norm,y_:trainY})
        cost = sess.run(cost_function,feed_dict={x:train_norm,y_:trainY})
        cost_history = np.append(cost_history,cost)
        correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y_,1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        accuracy = (sess.run(accuracy,feed_dict={x:test_norm,y_:testY}))
        #print ('Accuracy -- >',accuracy)
        if epoch % 200 == 0:
            print('epoch -> ',epoch,'cost -->',cost,'accuracy -->',accuracy)

epoch ->  0 cost --> 12.480889 accuracy --> 0.0992
epoch ->  200 cost --> 14.987206 accuracy --> 0.0708
epoch ->  400 cost --> 12.18769 accuracy --> 0.0986
epoch ->  600 cost --> 18.866968 accuracy --> 0.0998
epoch ->  800 cost --> 15.08289 accuracy --> 0.0796
epoch ->  1000 cost --> 17.265402 accuracy --> 0.0957
epoch ->  1200 cost --> 12.070403 accuracy --> 0.0998
epoch ->  1400 cost --> 11.157529 accuracy --> 0.0715
epoch ->  1600 cost --> 16.649942 accuracy --> 0.113
epoch ->  1800 cost --> 16.623291 accuracy --> 0.0847


In [0]:
 # # Training for Perceptron 

In [0]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features

In [0]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_dim])
y_ = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))

In [0]:
# Parameters
learning_rate = 0.0001
training_epochs = 5000
cost_history = np.empty(shape=[1],dtype=float)
n_dim = train_norm.shape[1]
n_class = 10 
batch_size = 100
display_step = 1

In [0]:
# Create model
def singlelayer_perceptron(x, weights, biases):
    # Hidden layer 
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    
    # Output layer with linear activation
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [0]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_class]))
}
biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}

In [0]:
# Construct model
pred = singlelayer_perceptron(x, weights, biases)

In [0]:
# Define loss and optimizer
cost_function = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost_function)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Launch the graph
with tf.Session() as sess:
    for epoch in range(training_epochs):
        sess.run(tf.global_variables_initializer())
        sess.run(optimizer,feed_dict={x:train_norm,y_:trainY})
        cost = sess.run(cost_function,feed_dict={x:train_norm,y_:trainY})
        cost_history = np.append(cost_history,cost)
        correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y_,1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        accuracy = (sess.run(accuracy,feed_dict={x:test_norm,y_:testY}))
        #print ('Accuracy -- >',accuracy)
        if epoch % 200 == 0:
            print('epoch -> ',epoch,'cost -->',cost,'accuracy -->',accuracy)

epoch ->  0 cost --> 58.861935 accuracy --> 0.053
epoch ->  200 cost --> 49.657993 accuracy --> 0.0732
epoch ->  400 cost --> 23.42132 accuracy --> 0.0826
epoch ->  600 cost --> 45.585373 accuracy --> 0.0884
epoch ->  800 cost --> 47.489532 accuracy --> 0.0153
epoch ->  1000 cost --> 66.80616 accuracy --> 0.1019
epoch ->  1200 cost --> 41.860115 accuracy --> 0.1214
epoch ->  1400 cost --> 47.296833 accuracy --> 0.0506
epoch ->  1600 cost --> 41.908028 accuracy --> 0.09
epoch ->  1800 cost --> 27.51727 accuracy --> 0.064
epoch ->  2000 cost --> 45.600445 accuracy --> 0.0848
epoch ->  2200 cost --> 38.3508 accuracy --> 0.1349
epoch ->  2400 cost --> 67.690704 accuracy --> 0.1255
epoch ->  2600 cost --> 51.667694 accuracy --> 0.1498
epoch ->  2800 cost --> 59.045734 accuracy --> 0.0798
epoch ->  3000 cost --> 59.798508 accuracy --> 0.1117
epoch ->  3200 cost --> 68.00215 accuracy --> 0.1319
epoch ->  3400 cost --> 60.970547 accuracy --> 0.0638
epoch ->  3600 cost --> 42.49562 accuracy -->

*****************************************************************  THE END ***************************************************************** 